### Probabilistic classification: RF & adjust tuning step


Thresholding in the context of:

1) Compute the difference between p(class) - t(class) **

1) Sorted in descending order

2) Compare max to threshold, if less then we move on to compare the next most likely class against the threshold

3) Continue until probability is equal or greater than threshold, assign that class

4) If no classes get assigned, then that cell is labelled as 'ambiguous

In [2]:
## Importing libraries 

import pandas as pd
import numpy as np 
import random 

from sklearn import preprocessing 
from statistics import mean

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA 

from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedRandomForestClassifier


from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 
from pprint import pprint 

from sklearn.preprocessing import normalize

import matplotlib.pyplot as plt 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve

#import ml_insights as mli 

### Importing data 

Require: 
1. input_files.txt - to contian filenames I want to use. ** currently .csv files

In [2]:
### 1) Importing annotated cells 
#Variables: mylist, inputs 

## obtaining list of files 
with open("D:/Tanrada_classification/imbalance_cortical_training/annotated/annotated_inputs_SD.txt") as f: 
    mylist= f.read().splitlines()
    
print("Read in: ",len(mylist),"files")

## 2) reading in all those files 
inputs = [] 
for i in mylist: 
    dat = pd.read_csv('D:/Tanrada_classification/imbalance_cortical_training/annotated/'+i,sep=",")
    ## Changing column names - since these names tend to be inconsistent causing problems 
    dat.columns.values[5] = 'Centroid_X'
    dat.columns.values[6] = 'Centroid_Y'
    dat.columns.values[8] = 'Nucleus: Area ¬µm^2'
    dat.columns.values[9] = 'Nucleus: Length ¬µm'
    dat.columns.values[12] = 'Nucleus: Max diameter ¬µm'
    dat.columns.values[13] = 'Nucleus: Min diameter ¬µm'
    dat.columns.values[14] = 'Cell: Area ¬µm^2'
    dat.columns.values[15] = 'Cell: Length ¬µm'
    dat.columns.values[18] = 'Cell: Max diameter ¬µm'
    dat.columns.values[19] = 'Cell: Min diameter ¬µm'
    #dat_cleaned = dat.iloc[:,0:61] ## SELECTING ONLY RELELVANT COLUMNS 
    print(i," number of features: ", dat.shape[1])
    inputs.append(dat)

print("Extracted:", len(inputs),"files")
print("Note: inconsistencies come from tau necrosis - will be removed later")
#Example
inputs[1].head()


Read in:  12 files
703484_cell_annotations_SD.csv  number of features:  342
721701_cell_annotations_SD.csv  number of features:  342
721735_cell_annotations_SD.csv  number of features:  342
721771_cell_annotations.csv  number of features:  62
721856_cell_annotations_SD.csv  number of features:  342
722215_cell_annotations_SD.csv  number of features:  342
755472_cell_annotations_SD.csv  number of features:  61
755480_cell_annotations.csv  number of features:  61
755481_cell_annotations.csv  number of features:  61
755485_cell_annotations_SD.csv  number of features:  61
755486_cell_annotations_SD.csv  number of features:  61
755525_cell_annotations_SD.csv  number of features:  61
Extracted: 12 files
Note: inconsistencies come from tau necrosis - will be removed later


,Image,Name,Class,Parent,ROI,Centroid_X,Centroid_Y,Detection probability,Nucleus: Area ¬µm^2,Nucleus: Length ¬µm,...,Smoothed: 50 µm: DAB: Membrane: Min,Smoothed: 50 µm: DAB: Membrane: Max,Smoothed: 50 µm: DAB: Membrane: Std.Dev.,Smoothed: 50 µm: DAB: Cell: Mean,Smoothed: 50 µm: DAB: Cell: Median,Smoothed: 50 µm: DAB: Cell: Min,Smoothed: 50 µm: DAB: Cell: Max,Smoothed: 50 µm: DAB: Cell: Std.Dev.,Smoothed: 50 µm: tau: Necrosis area µm^2,Smoothed: 50 µm: Nearby detection counts
0,721701.svs,Grey_matter,Epithelial_new,PathAnnotationObject,Polygon,13262.7,12570.4,0.8110,19.2875,17.1735,...,-0.0264,0.2883,0.0551,0.0567,0.0372,-0.0725,0.4021,0.0615,10.1183,7
1,721701.svs,Grey_matter,Epithelial_new,PathAnnotationObject,Polygon,13281.1,12573.9,0.7445,15.8931,16.4348,...,-0.0239,0.2995,0.0571,0.0617,0.0392,-0.0673,0.4383,0.0686,11.9707,8
2,721701.svs,Grey_matter,Epithelial_new,PathAnnotationObject,Polygon,13258.4,12577.8,0.8102,42.2255,25.9681,...,-0.0265,0.2878,0.0552,0.0553,0.0372,-0.0722,0.3911,0.0590,9.5437,7
3,721701.svs,Grey_matter,Oligo_new,PathAnnotationObject,Polygon,13328.6,12592.3,0.8895,22.8011,17.0965,...,-0.0308,0.2705,0.0568,0.0602,0.0405,-0.0937,0.3983,0.0655,13.3102,11
4,721701.svs,Grey_matter,Oligo_new,PathAnnotationObject,Polygon,13296.7,12622.4,0.8204,17.3133,15.4801,...,-0.0233,0.2791,0.0484,0.0588,0.0432,-0.0866,0.4009,0.0596,10.8676,9


In [3]:
# 2.5) Importing in neghbouring cells info (numbers)

with open("D:/Tanrada_classification/number_NNB/to_use/imbalance_NB_inputs_SD.txt") as f: 
    nb_mylist= f.read().splitlines()
    
print("Read in:",len(nb_mylist)," NUMBER OF neighbouring cells files")

# reading in all those files 
nb_inputs = [] 
for i in nb_mylist: 
    dat = pd.read_csv("D:/Tanrada_classification/number_NNB/to_use/"+i,sep=",") 
    nb_inputs.append(dat)
    
print("Extracted:", len(nb_inputs),"files")

Read in: 12  NUMBER OF neighbouring cells files
Extracted: 12 files


In [4]:
# Inspecting number of columns in NNB files: 
for i in nb_inputs:
    print("Number of features ",i.shape[1])

Number of features  13
Number of features  13
Number of features  13
Number of features  13
Number of features  13
Number of features  13
Number of features  13
Number of features  13
Number of features  13
Number of features  13
Number of features  13
Number of features  13


In [5]:
## Extracting only annotated cells from nb files 
inputs_=[]
n=len(inputs)
for i in range(0,n):
    #all cells on slide
    nb_dat = nb_inputs[i]
    nb_dat = nb_dat.rename(columns={'X':'Centroid_X','Y':'Centroid_Y'})
    print("nb_dat shape:", nb_dat.shape)
    #annotated cells with no nb info
    dat = inputs[i]
    print("dat shape:", dat.shape)
    #annotated cells with nb info: intersect between 2 dataframes 
    combined = dat.merge(nb_dat,on=['Centroid_X','Centroid_Y'],how='inner',validate='1:1') 
    inputs_.append(combined)
    print("Expected shape:", dat.shape[0],dat.shape[1]+nb_dat.shape[1]-2," Resulting shape:",combined.shape)
    print("--------------------------")
    
print("Succesfully combined nb cell counts to main data")

nb_dat shape: (485605, 13)
dat shape: (573, 342)
Expected shape: 573 353  Resulting shape: (573, 353)
--------------------------
nb_dat shape: (272349, 13)
dat shape: (193, 342)
Expected shape: 193 353  Resulting shape: (193, 353)
--------------------------
nb_dat shape: (410302, 13)
dat shape: (253, 342)
Expected shape: 253 353  Resulting shape: (253, 353)
--------------------------
nb_dat shape: (413521, 13)
dat shape: (197, 62)
Expected shape: 197 73  Resulting shape: (197, 73)
--------------------------
nb_dat shape: (442130, 13)
dat shape: (216, 342)
Expected shape: 216 353  Resulting shape: (216, 353)
--------------------------
nb_dat shape: (475306, 13)
dat shape: (246, 342)
Expected shape: 246 353  Resulting shape: (246, 353)
--------------------------
nb_dat shape: (357603, 13)
dat shape: (625, 61)
Expected shape: 625 72  Resulting shape: (625, 72)
--------------------------
nb_dat shape: (516132, 13)
dat shape: (593, 61)
Expected shape: 593 72  Resulting shape: (593, 72)
----

In [6]:
### 2) Importing hema nucleus mean of all detected cells & location coordinates 
# Variables: hema_mylist, hema_inputs 

## obtaining list of files 
with open("D:/Tanrada_classification/imbalance_cortical_training/hema/annotated_hema_SD.txt") as f: 
    hema_mylist= f.read().splitlines()
    
print("Read in:",len(hema_mylist),"hema files")    


## 4) reading in all those files 
hema_inputs = [] 
for i in hema_mylist: 
    dat = pd.read_csv('D:/Tanrada_classification/imbalance_cortical_training/hema/'+i,sep=",")
    dat.columns.values[0] = 'Centroid_X' # To fix naming inconsistency problem 
    dat.columns.values[1] = 'Centroid_Y'
    hema_inputs.append(dat)

print("Extracted:",len(hema_inputs),"hema files")  

#Example 
hema_inputs[2].head()


Read in: 12 hema files
Extracted: 12 hema files


,Centroid_X,Centroid_Y,Hematoxylin: Nucleus: Mean
0,8351.9,478.67,0.1949
1,8250.8,479.90,0.1938
2,8300.6,479.33,0.2000
3,8294.8,487.18,0.2580
4,8230.7,497.89,0.2701


In [7]:
# Checking if filenames & order of them from mylist, nb_mylist & hema_mylist match
x_nb = [i[1:7] for i in nb_mylist]
x = [i[0:6] for i in mylist]
x_h = [i[0:6] for i in hema_mylist]
print("mylist & nb_list matched?:", x==x_nb)
print("mylist & hema_list matched?:",x==x_h)

mylist & nb_list matched?: True
mylist & hema_list matched?: True


### Normalising hematoxlyin per brain side & discard top 1%

In [8]:
### 1) Get instances needed to be remove for each slide
#Variables: hema_to_remove, hema_inputs 
hema_to_remove = [] 
for h in hema_inputs: 
    h2 = h.copy() 
    hema = h2['Hematoxylin: Nucleus: Mean']
    threshold = hema.quantile(0.99)
    hema_norm = hema/threshold 
    h2['Hematoxylin: Nucleus: Mean'] = hema_norm 
    h2 = h2[h2['Hematoxylin: Nucleus: Mean']>1] # to select instances need removing (keep <=1)
    hema_to_remove.append(h2)

for i in range(0,len(hema_to_remove)): 
    print(i, " No. of cells with normalised Hema >1:",len(hema_to_remove[i]), "from", len(hema_inputs[i]),"detected cells")

0  No. of cells with normalised Hema >1: 4848 from 485605 detected cells
1  No. of cells with normalised Hema >1: 2720 from 272349 detected cells
2  No. of cells with normalised Hema >1: 4104 from 410302 detected cells
3  No. of cells with normalised Hema >1: 4132 from 413521 detected cells
4  No. of cells with normalised Hema >1: 4415 from 442130 detected cells
5  No. of cells with normalised Hema >1: 4736 from 475306 detected cells
6  No. of cells with normalised Hema >1: 3577 from 357603 detected cells
7  No. of cells with normalised Hema >1: 5144 from 516132 detected cells
8  No. of cells with normalised Hema >1: 2964 from 296324 detected cells
9  No. of cells with normalised Hema >1: 3314 from 331459 detected cells
10  No. of cells with normalised Hema >1: 5301 from 530977 detected cells
11  No. of cells with normalised Hema >1: 3642 from 364677 detected cells


In [9]:
## 2) Discarding annotated cells if they fit the criteria above 
#Variables: cleaned_inputs, removed  

cleaned_inputs = []
removed = [] 
for n in range(0,(len(inputs_))): #looping through annotated % hema (detected) slides 
    
    i = inputs_[n] #annotated cells 
    h = hema_to_remove[n] #cells we need to remove, may or may not contain annotated cells 
    
    #Find cells that exist in both 'i' & 'h' = cells we want to remove 
    to_remove = i.merge(h,on=['Centroid_X','Centroid_Y'],how='inner',validate='1:1')
    
    #Find cells that only exist in 'i' but not in 'h' = cells we want to retain 
    to_retain = i.merge(h,on=['Centroid_X','Centroid_Y'],how='left',indicator=True,validate='1:1')
    
    #Extract cells we want to retain 
    retained = i[to_retain['_merge']=='left_only']
    
    cleaned_inputs.append(retained)
    removed.append(to_remove)
    print(mylist[n],":",i.shape[0]-retained.shape[0],"cells removed")


703484_cell_annotations_SD.csv : 7 cells removed
721701_cell_annotations_SD.csv : 3 cells removed
721735_cell_annotations_SD.csv : 5 cells removed
721771_cell_annotations.csv : 6 cells removed
721856_cell_annotations_SD.csv : 1 cells removed
722215_cell_annotations_SD.csv : 2 cells removed
755472_cell_annotations_SD.csv : 2 cells removed
755480_cell_annotations.csv : 5 cells removed
755481_cell_annotations.csv : 4 cells removed
755485_cell_annotations_SD.csv : 0 cells removed
755486_cell_annotations_SD.csv : 3 cells removed
755525_cell_annotations_SD.csv : 1 cells removed


### Checking features

In [10]:
print(cleaned_inputs[0].shape)
list(cleaned_inputs[0].columns)

(566, 353)


['Image',
 'Name',
 'Class',
 'Parent',
 'ROI',
 'Centroid_X',
 'Centroid_Y',
 'Detection probability',
 'Nucleus: Area ¬µm^2',
 'Nucleus: Length ¬µm',
 'Nucleus: Circularity',
 'Nucleus: Solidity',
 'Nucleus: Max diameter ¬µm',
 'Nucleus: Min diameter ¬µm',
 'Cell: Area ¬µm^2',
 'Cell: Length ¬µm',
 'Cell: Circularity',
 'Cell: Solidity',
 'Cell: Max diameter ¬µm',
 'Cell: Min diameter ¬µm',
 'Nucleus/Cell area ratio',
 'Hematoxylin: Nucleus: Mean',
 'Hematoxylin: Nucleus: Median',
 'Hematoxylin: Nucleus: Min',
 'Hematoxylin: Nucleus: Max',
 'Hematoxylin: Nucleus: Std.Dev.',
 'Hematoxylin: Cytoplasm: Mean',
 'Hematoxylin: Cytoplasm: Median',
 'Hematoxylin: Cytoplasm: Min',
 'Hematoxylin: Cytoplasm: Max',
 'Hematoxylin: Cytoplasm: Std.Dev.',
 'Hematoxylin: Membrane: Mean',
 'Hematoxylin: Membrane: Median',
 'Hematoxylin: Membrane: Min',
 'Hematoxylin: Membrane: Max',
 'Hematoxylin: Membrane: Std.Dev.',
 'Hematoxylin: Cell: Mean',
 'Hematoxylin: Cell: Median',
 'Hematoxylin: Cell: Min',

### Removing DAB & tau necrosis & Image_name

In [11]:
## Removing DAB & tau necrosis ** making sure same dimension
cleaned_inputs_ = []
for i in cleaned_inputs:
    # To remove all DAB features
    to_drop1 = list(i.filter(regex='DAB'))
    dat1 = i[i.columns.drop(to_drop1)]
    # To remove tau necrosis features
    to_drop2 = list(dat1.filter(regex='tau'))
    dat2= dat1[dat1.columns.drop(to_drop2)]
    # To remove Smoothed ****
    to_drop3 = list(dat2.filter(regex='Smoothed'))
    dat3= dat2[dat2.columns.drop(to_drop3)]
    #Remove Image_name
    dat = dat3.drop(columns=['Image_name'])
    
    cleaned_inputs_.append(dat)
    print("Initial n_features:",i.shape[1], ", After removal:", dat.shape[1])

Initial n_features: 353 , After removal: 51
Initial n_features: 353 , After removal: 51
Initial n_features: 353 , After removal: 51
Initial n_features: 73 , After removal: 51
Initial n_features: 353 , After removal: 51
Initial n_features: 353 , After removal: 51
Initial n_features: 72 , After removal: 51
Initial n_features: 72 , After removal: 51
Initial n_features: 72 , After removal: 51
Initial n_features: 72 , After removal: 51
Initial n_features: 72 , After removal: 51
Initial n_features: 72 , After removal: 51


### Putting the slides together 

In [12]:
##Variables: labelled_orig, labelled_data 
#1) Put the slides together

labelled_orig = pd.concat(cleaned_inputs_)
print(labelled_orig.shape)

# 2) Extract relevant columns 
dat = labelled_orig.drop(columns=['Name','Parent','ROI']) 
dat.head()


(4717, 51)


,Image,Class,Centroid_X,Centroid_Y,Detection probability,Nucleus: Area ¬µm^2,Nucleus: Length ¬µm,Nucleus: Circularity,Nucleus: Solidity,Nucleus: Max diameter ¬µm,...,NN_10_um,NN_20_um,NN_30_um,NN_40_um,NN_50_um,NN_60_um,NN_70_um,NN_80_um,NN_90_um,NN_100_um
0,703484.svs,fragmented,2758.4,5064.2,0.7102,10.4438,11.7588,0.9492,1.0000,4.2279,...,1,2,3,5,7,10,16,21,25,36
1,703484.svs,Epithelial,2865.6,5210.4,0.7737,34.4719,24.9365,0.6966,0.9739,10.6183,...,0,1,3,3,4,7,14,25,32,35
2,703484.svs,Epithelial,2929.0,5210.6,0.7337,23.0844,20.5225,0.6888,0.9821,8.7795,...,0,0,1,2,5,5,10,15,18,25
3,703484.svs,Ignore,2736.6,5224.5,0.5949,31.9419,20.3826,0.9662,1.0000,7.1633,...,0,0,0,1,7,13,15,20,28,35
4,703484.svs,fragmented,5262.3,5460.8,0.7159,9.8636,13.0301,0.7300,0.9790,5.4442,...,1,4,7,8,13,16,17,19,22,30


In [13]:
list(dat.columns)

['Image',
 'Class',
 'Centroid_X',
 'Centroid_Y',
 'Detection probability',
 'Nucleus: Area ¬µm^2',
 'Nucleus: Length ¬µm',
 'Nucleus: Circularity',
 'Nucleus: Solidity',
 'Nucleus: Max diameter ¬µm',
 'Nucleus: Min diameter ¬µm',
 'Cell: Area ¬µm^2',
 'Cell: Length ¬µm',
 'Cell: Circularity',
 'Cell: Solidity',
 'Cell: Max diameter ¬µm',
 'Cell: Min diameter ¬µm',
 'Nucleus/Cell area ratio',
 'Hematoxylin: Nucleus: Mean',
 'Hematoxylin: Nucleus: Median',
 'Hematoxylin: Nucleus: Min',
 'Hematoxylin: Nucleus: Max',
 'Hematoxylin: Nucleus: Std.Dev.',
 'Hematoxylin: Cytoplasm: Mean',
 'Hematoxylin: Cytoplasm: Median',
 'Hematoxylin: Cytoplasm: Min',
 'Hematoxylin: Cytoplasm: Max',
 'Hematoxylin: Cytoplasm: Std.Dev.',
 'Hematoxylin: Membrane: Mean',
 'Hematoxylin: Membrane: Median',
 'Hematoxylin: Membrane: Min',
 'Hematoxylin: Membrane: Max',
 'Hematoxylin: Membrane: Std.Dev.',
 'Hematoxylin: Cell: Mean',
 'Hematoxylin: Cell: Median',
 'Hematoxylin: Cell: Min',
 'Hematoxylin: Cell: Max',


### Extracting relevant cell classes

In [14]:
# 1) Check no. of cells / class of our data
print("Total",sum(dat['Class'].value_counts()),"cells")
dat['Class'].value_counts()


Total 4717 cells


Oligo_New         461
Neuron_new        445
Neuron_New        440
Ignore            356
Oligo_new         331
Epithelial        330
Epithelial_new    310
fragmented        304
Neuron            302
Astro             297
Oligo             295
Epithelial_New    247
Ignore_New        201
Ignore_new        143
Astro_New          97
Astro_new          82
Stardist_error     35
Tumor              16
fragmented_new     15
Fragmented_New     10
Name: Class, dtype: int64

In [15]:
# 2) Making all class names lower case - easier for later selection.

dat_lower_class = [i.lower() for i in dat['Class']]
dat__ = dat.copy()
dat__['Class'] = dat_lower_class 
dat__['Class'].value_counts()

neuron_new        885
oligo_new         792
epithelial_new    557
ignore            356
ignore_new        344
epithelial        330
fragmented        304
neuron            302
astro             297
oligo             295
astro_new         179
stardist_error     35
fragmented_new     25
tumor              16
Name: Class, dtype: int64

In [16]:
# 3) Selecting only relevant cell classes (Using stardist_error instead of ignore_new)
orig = dat__.copy()
dat_ = dat__[(dat__['Class'] == 'oligo_new') | (dat__['Class'] == 'neuron_new')
          | (dat__['Class'] == 'astro_new')| (dat__['Class'] == 'epithelial_new')
          | (dat__['Class'] == 'stardist_error')| (dat__['Class'] == 'fragmented_new')]
dat_=dat_.reset_index(drop=True)

In [17]:
# 4) Checking results from 3)
dat_['Class'].value_counts()

neuron_new        885
oligo_new         792
epithelial_new    557
astro_new         179
stardist_error     35
fragmented_new     25
Name: Class, dtype: int64

In [18]:
# 5) Re-name these classes so it has no '_new'
class_new = dat_['Class']
x = [(i[0:-4].capitalize()) for i in class_new]
#dat = dat_ 
dat_['Class'] = x
dat_['Class'].value_counts()

Neuron        885
Oligo         792
Epithelial    557
Astro         179
Stardist_e     35
Fragmented     25
Name: Class, dtype: int64

In [19]:
# 6) Group Ignore, Epithelial & Fragmented cells as a single class called 'Others'
class_ = dat_['Class']
y = ['Others'if i == 'Epithelial' or i == 'Stardist_e' or i == 'Fragmented' else i for i in class_new ]
dat = dat_
dat['Class'] = y 
print(dat['Class'].value_counts().sum())
dat['Class'].value_counts()


2473


Neuron    885
Oligo     792
Others    617
Astro     179
Name: Class, dtype: int64

In [20]:
#if cv=3
dat['Class'].value_counts()/3

Neuron    295.000000
Oligo     264.000000
Others    205.666667
Astro      59.666667
Name: Class, dtype: float64

In [21]:
#if cv=5 
dat['Class'].value_counts()/5

Neuron    177.0
Oligo     158.4
Others    123.4
Astro      35.8
Name: Class, dtype: float64

# Training the model

### Checking for any NA in the data

In [22]:
#checking for NAN 
## NEW 
print("Any NA in the data?: ",dat.isnull().sum().sum()==1)

#dat = dat.dropna()
#dat.isnull().sum().sum()

Any NA in the data?:  False


### Create train, test sets 

In [23]:
#We are using the entire dataset to train the model, test data will be provided later by Sanne 
X_train_l = dat.drop(columns=['Class'])
X_train = X_train_l.drop(columns=['Image','Centroid_X','Centroid_Y'])
print('training data shape:',X_train_l.shape)
y_train = dat['Class']

training data shape: (2473, 47)


In [24]:
X_train.head()

,Detection probability,Nucleus: Area ¬µm^2,Nucleus: Length ¬µm,Nucleus: Circularity,Nucleus: Solidity,Nucleus: Max diameter ¬µm,Nucleus: Min diameter ¬µm,Cell: Area ¬µm^2,Cell: Length ¬µm,Cell: Circularity,...,NN_10_um,NN_20_um,NN_30_um,NN_40_um,NN_50_um,NN_60_um,NN_70_um,NN_80_um,NN_90_um,NN_100_um
0,0.8929,70.4335,30.1589,0.9731,1.0000,10.5379,8.6989,258.8036,60.0120,0.9030,...,0,2,3,7,9,12,14,18,28,36
1,0.6650,69.3602,30.0618,0.9645,1.0000,10.3145,8.9289,289.4437,60.9183,0.9801,...,0,3,4,8,8,9,17,22,27,35
2,0.9025,21.2553,16.7824,0.9483,1.0000,6.1317,4.5305,132.4307,42.3046,0.9299,...,0,2,4,5,10,13,15,18,22,35
3,0.7240,91.2679,34.9913,0.9367,0.9992,12.8068,8.8117,321.6940,65.4879,0.9426,...,0,2,2,5,8,14,19,26,33,40
4,0.8682,18.5580,15.5796,0.9608,1.0000,5.4150,4.4888,163.2746,45.8910,0.9743,...,0,0,1,7,12,15,17,22,28,37


### My own functions 

In [25]:
## Functions for custom classification metrics 

## Accuracy per class 
def astro_acc(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    acc_c = cm.diagonal()
    return acc_c[0] #astrocytes acc

def neuron_acc(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    acc_c = cm.diagonal()
    return acc_c[1] #neuron acc

def oligo_acc(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    acc_c = cm.diagonal()
    return acc_c[2] #oligo acc

def others_acc(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    acc_c = cm.diagonal()
    return acc_c[3] #ignore acc



## Confusion per class: 

## Astro
def A_as_N(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[0][1] # percentage that A is wrongly classified as N   

def A_as_O(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[0][2] # percentage that A is wrongly classified as O       


def A_as_Others(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[0][3]   

##Neurons 

def N_as_A(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[1][0] 

def N_as_O(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[1][2] 

def N_as_Others(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[1][3] 


## Oligo 

def O_as_A(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[2][0] 

def O_as_N(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[2][1] 

def O_as_Others(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[2][3] 

## Others 

def Others_as_A(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[3][0] 

def Others_as_N(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[3][1] 

def Others_as_O(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"],normalize='true')
    return cm[3][2] 


In [26]:
## Functions for custom classification metrics: RAW VALUES 


## Confusion per class: 

## Astro
def A_as_N_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[0][1] # percentage that A is wrongly classified as N   

def A_as_O_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[0][2] # percentage that A is wrongly classified as O       


def A_as_Others_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[0][3]   

##Neurons 

def N_as_A_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[1][0] 

def N_as_O_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[1][2] 

def N_as_Others_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[1][3] 


## Oligo 

def O_as_A_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[2][0] 

def O_as_N_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[2][1] 

def O_as_Others_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[2][3] 

## Others 

def Others_as_A_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[3][0] 

def Others_as_N_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[3][1] 

def Others_as_O_r(clf,X,y): 
    y_pred = clf.predict(X)
    cm = confusion_matrix(y,y_pred,labels=["Astro","Neuron","Oligo","Others"])
    return cm[3][2] 


In [27]:
### Precision-recall score
def precision_recall_auc(clf,X,y):
    #Variables
    pr_score={}
    
    #get y prob predictions
    y_prob_pred = clf.predict_proba(X)
    
    #Convert true y name into numerical classes
    y_true_numeric = name_to_numeric_classes(y)
    
    #get number of classes
    n_class = list(set(y))
    
    #create PR curve using OVR approach 
    for i in range(len(n_class)): # for each class, calculate roc_curve 
        p, r, thresh = precision_recall_curve(y_true_numeric, y_prob_pred[:,i], pos_label=i)
        pr_score[i] = auc(r,p) #recall on x axis, precision on y axis
        
    #Combine all pr-scores using 'macro' method
    pr_auc = mean(pr_score.values())
    return pr_auc
        

In [28]:
# ADDITIONAL FUNCTIONS

#FUNCTIONS

# Get all class-specific thresholds from best params

def get_threshold(best_params):
    class_thresholds=[]
    for i in best_params:
        t = best_params[i][0]
        class_thresholds.append(t)
    return class_thresholds 

#Thresholding method 5: using ratio, ambiguous cells are: 
# 1) When predicted probabilities < thresholds, i.e. diff = -ve (lower end)
# 2) When more than 1 class-specific threshold is passed, i.e. more than 1 positive diff scores (upper end)

def threshold_list_of_classes_5(y_pred_prob,best_params):
    
    thresholded_classes=[]
    for i in y_pred_prob: #for each cell (containing 4 class probabilities)
        
        #Get cell 4 class-specific threshold values: 
        thresholds = get_threshold(best_params)
        
        # Calculate predicted probability - threshold = difference for each of the 4 classes 
        differences = (i-thresholds)/thresholds
        
        #Count number of positive or equal (0) differences 
        count = np.count_nonzero(differences>=0)
        
        if (count==1): #only assign class when 1 class passes the threshold 
            pred_class = np.argmax(differences)
        else: #Otherwise, label as ambiguous (when more than 1 class passes, or when no class passes)
            pred_class=4

        #putting prediction in a list
        thresholded_classes.append(pred_class)

    thresholded_classes_ = numeric_to_name_classes(thresholded_classes)
    return thresholded_classes_
             
#Thresholding method 4  using ratio instead of raw difference
def threshold_list_of_classes_4 (y_pred_prob,best_params):
    
    thresholded_classes=[]
    for i in y_pred_prob: #for each cell (containing 4 class probabilities)
        
        #Get cell 4 class-specific threshold values: 
        thresholds = get_threshold(best_params)
        
        # Calculate predicted probability - threshold = difference for each of the 4 classes 
        differences = (i-thresholds)/thresholds
        
        #Check if there is at last one positive diff probability
        if (np.any(differences>0)): # If true, 
            
            #get index (indicative of class) of the highest probability difference
            pred_class = np.argmax(differences) 
            
        else: #If all diff probabilities are NEGATIVE
            pred_class = 4 # Assign cell class as 'ambiguous'

        #putting prediction in a list
        thresholded_classes.append(pred_class)

    thresholded_classes_ = numeric_to_name_classes(thresholded_classes)
    return thresholded_classes_


#Thresholding method 3)

def threshold_list_of_classes_3 (y_pred_prob,best_params):
    
    thresholded_classes=[]
    for i in y_pred_prob: #for each cell (containing 4 class probabilities)
        
        #Get cell 4 class-specific threshold values: 
        thresholds = get_threshold(best_params)
        
        # Calculate predicted probability - threshold = difference for each of the 4 classes 
        differences = i-thresholds
        
        #Check if there is at last one positive diff probability
        if (np.any(differences>0)): # If true, 
            
            #get index (indicative of class) of the highest probability difference
            pred_class = np.argmax(differences) 
            
        else: #If all diff probabilities are NEGATIVE
            pred_class = 4 # Assign cell class as 'ambiguous'

        #putting prediction in a list
        thresholded_classes.append(pred_class)

    thresholded_classes_ = numeric_to_name_classes(thresholded_classes)
    return thresholded_classes_

#Thresholding method 2)
def threshold_list_of_classes_2(y_pred_prob,best_params):
    
    thresholded_classes=[]
    for i in y_pred_prob: #for each cell 
        
        #Get threshold values: 
        thresholds = get_threshold(best_params)
        
        # Calculate predicted probability - threshold = difference 
        differences = i-thresholds
               
        #get index (indicative of class) of the highest probability
        pred_class = np.argmax(differences)         

        #putting prediction in a list
        thresholded_classes.append(pred_class)

        
    thresholded_classes_ = numeric_to_name_classes(thresholded_classes)
    return thresholded_classes_


# To convert numeric classes to its corresponding name classes 
def numeric_to_name_classes(numeric_classes):
    output=[]
    for i in numeric_classes:
        if (i==0):
            c = 'Astro'
        elif(i==1):
            c='Neuron'
        elif(i==2):
            c='Oligo'
        elif(i==3):
            c='Others'
        elif(i==4):
            c='Ambiguous'
        else:
            print('SOMETHING IS WRONG')
        output.append(c)
    return output

# To convert name classes to its corresponding numeric classes: 
def name_to_numeric_classes(name_classes):
    output=[]
    for i in name_classes: 
        if (i == 'Astro'): 
            x=0
        elif(i == 'Neuron'): 
            x=1
        elif(i == 'Oligo'): 
            x=2
        elif(i=='Others'):
            x=3
        elif(i=='Ambiguous'):
            x=4
        else:
            print('SOMETHING IS WRONG')
            break
        output.append(x)
    return output
    
# Create roc curve for each class in multi-classification problem

def multiclass_roc_curves(n_class,test_y_numeric,predy):
    
    fpr = {}
    tpr = {}
    thresh ={}
    
    #calcualte roc curve locations 
    for i in range(n_class): # for each class, calculate roc_curve 
        fpr[i], tpr[i], thresh[i] = roc_curve(test_y_numeric, predy[:,i], pos_label=i)
    
    return fpr,tpr,thresh

#Find the best position on the roc curve for multi-classification problem
def best_param_gmean(n_class,fpr,tpr,thresh):
    #calculate g-mean for each threshold 
    #gmeans={}
    best_params={}
    class_names=['Astro','Neuron','Oligo','Others']
    for i in range(n_class):
        tpr_ = tpr[i]
        fpr_ = fpr[i]
        gm = np.sqrt(tpr_*(1-fpr_))
        t = thresh[i]
        #gmeans[i]=gm
        ix = np.argmax(gm)
        best_params[i] = (t[ix],gm[ix],fpr_[ix],tpr_[ix])
        #print(gm)
       # print(class_names[i],'Best Threshold=%f, G-Mean=%.3f, fpr=%f, tpr=%f' % (t[ix], gm[ix],fpr_[ix],tpr_[ix]))
        #print('--------------------------------------------------')
    return best_params

def multiclass_PR_curves(n_class,test_y_numeric,predy):
    
    precision = {}
    recall = {}
    thresh ={}
    
    #calcualte roc curve locations 
    for i in range(n_class): # for each class, calculate roc_curve 
        precision[i], recall[i], thresh[i] = precision_recall_curve(test_y_numeric, predy[:,i], pos_label=i)
    
    return precision,recall,thresh

#Find the best position on the roc curve for multi-classification problem
def best_param_f_score(n_class,precision,recall,thresh):
    #calculate g-mean for each threshold 
    #f_scores={}
    best_params={}
    class_names=['Astro','Neuron','Oligo','Others']
    for i in range(n_class):
        p = precision[i]
        r = recall[i]
        nu=(2*p*r)
        de=(p+r) 
        f_score = np.divide(nu,de,out=np.zeros_like(nu),where=de != 0) #(2*p*r)/(p+r)
        t = thresh[i]
        #f_scores[i]=f_score
        ix = np.argmax(f_score)
        best_params[i] = (t[ix],f_score[ix],p[ix],r[ix])
        #print(gm)
       # print(class_names[i],'Best Threshold=%f, G-Mean=%.3f, fpr=%f, tpr=%f' % (t[ix], gm[ix],fpr_[ix],tpr_[ix]))
        #print('--------------------------------------------------')
    return best_params

#Thresholding function
def prob_thresholding(y_pred_prob,y_pred,threshold):
    thresholded_class =[]
    for i in range(0,len(y_pred_prob)):
        if(max(y_pred_prob[i])<threshold):
            c='Ambiguous'
        else:
            c=y_pred[i]
        thresholded_class.append(c)
    return thresholded_class

#Removing ambiguous class from thresholded class & y_predict
def remove_amb_class(t_class,y_test):
    
    #Get indices of instances with no ambiguous label 
    x = pd.Series(t_class)
    y_pred_no_amb = x[x!='Ambiguous']
    y_pred_no_amb_indices = y_pred_no_amb.index
    
    #Extract these instances fom y_pred
    #y_predict_no_amb = y_predict.iloc[pred_no_amb_indices]
    
    #Subset y_test
    y_test_no_amb = y_test.iloc[y_pred_no_amb_indices]
    
    return (y_pred_no_amb,y_test_no_amb)

### Hyperparameter tuning - random forest

In [29]:
pipeline = Pipeline([
    ('normalizer',MinMaxScaler()),
    ('selector',RFE(SVC(kernel='linear'))), 
    ('clf',BalancedRandomForestClassifier())
])
#pipeline.set_params(clf=RandomForestClassifier())
pipeline.steps

[('normalizer', MinMaxScaler()),
 ('selector', RFE(estimator=SVC(kernel='linear'))),
 ('clf', BalancedRandomForestClassifier())]

In [30]:
ccp_alphas = [float(x) for x in np.linspace(start=0, stop=0.03, num=7) ]
ccp_alphas

[0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03]

In [8]:
ccp_alphas = [float(x) for x in np.linspace(start=0, stop=0.03, num=7) ]#[float(x) for x in np.linspace(start=0, stop=1, num=10) ]
ccp_alphas

[0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03]

cv=10

In [3]:
### Hyper parameters to tune

#Number of trees in random forest 
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

#Number of features to consider at every split
max_features = ['auto', 'sqrt']

#Maximum number of levels in tree 
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

#Minimum number of samples required to split an internal node 
min_samples_split = [2, 5, 10]

#Minimum number of samples required at each leaf node 
min_samples_leaf = [1, 2, 4]

#Method for selecting samples for training each tree 
bootstrap = [True, False]

#sampling strategy
sampling_strategy=['auto','all','not majority','majority']

#ccp_alphas 
ccp_alphas = [float(x) for x in np.linspace(start=0, stop=0.03, num=7) ]#[float(x) for x in np.linspace(start=0, stop=1, num=10) ]

# ## Create the random grid 
# random_grid = {'selector__n_features_to_select':[30,32,34,36,38,40],
#                 'clf__n_estimators': n_estimators,
#                'clf__max_features': max_features,
#                'clf__max_depth': max_depth,
#                'clf__min_samples_split': min_samples_split,
#                'clf__min_samples_leaf': min_samples_leaf,
#               'clf__bootstrap': bootstrap,
#               'clf__random_state':[42],
#                'clf__sampling_strategy':sampling_strategy, 
#                'clf__ccp_alpha':ccp_alphas
#              # 'clf__class_weight':['balanced']
#               } # newly added
# #pprint(random_grid)

# rf_random = RandomizedSearchCV(pipeline,
#                              param_distributions=random_grid, 
#                              n_iter=100,
#                              cv=10,
#                              verbose=2,
#                             random_state=42,
#                             n_jobs=-1,
#                               refit='PR_AUC', # use this metric to evaluate performance of parameters 
#                       scoring={'PR_AUC':precision_recall_auc,
#                           'roc_auc_ovr_weighted':'roc_auc_ovr_weighted',
#                             'roc_auc_ovo':'roc_auc_ovo',
#                               'balanced_accuracy':'balanced_accuracy',
#                                'f1_weighted':'f1_weighted',
#                                'Astro_accuracy': astro_acc,
#                                'Neuron_accuracy':neuron_acc,
#                                'Oligo_accuracy':oligo_acc,
#                                'Others_accuracy':others_acc,
#                                'A_as_N':A_as_N,
#                                'A_as_O':A_as_O,
#                                'A_as_Others':A_as_Others,
#                                'N_as_A':N_as_A,
#                                'N_as_O':N_as_O,
#                                'N_as_Others':N_as_Others,
#                                'O_as_A':O_as_A,
#                                'O_as_N':O_as_N,
#                                'O_as_Others':O_as_Others,
#                                'Others_as_A':Others_as_A,
#                                'Others_as_N':Others_as_N,
#                                'Others_as_O':Others_as_O
#                               })

# rf_random.fit(X_train,y_train)

# print(rf_random.best_score_)
# print(rf_random.best_params_)


Fitting 10 folds for each of 100 candidates, totalling 1000 fits
0.8092402569869004
{'selector__n_features_to_select': 32, 'clf__sampling_strategy': 'all', 'clf__random_state': 42, 'clf__n_estimators': 1400, 'clf__min_samples_split': 2,
 'clf__min_samples_leaf': 4, 'clf__max_features': 'sqrt', 'clf__max_depth': 10, 'clf__bootstrap': False}

In [32]:
# # Digging into more details 
# print("PR-AUC:",
#      rf_random.cv_results_['mean_test_PR_AUC'][rf_random.best_index_]*100)
# print("ROC-AUC:",
#      rf_random.cv_results_['mean_test_roc_auc_ovr_weighted'][rf_random.best_index_]*100)
# print("ROC-AUC:",
#      rf_random.cv_results_['mean_test_roc_auc_ovo'][rf_random.best_index_]*100)

# print("Balanced accuracy:",
#       rf_random.cv_results_['mean_test_balanced_accuracy'][rf_random.best_index_]*100)

# print("F1_weighted:",
#       rf_random.cv_results_['mean_test_f1_weighted'][rf_random.best_index_]*100)

# print("Astrocyte accuracy:",
#       rf_random.cv_results_['mean_test_Astro_accuracy'][rf_random.best_index_]*100)

# print("Neuron accuracy:",
#       rf_random.cv_results_['mean_test_Neuron_accuracy'][rf_random.best_index_]*100)

# print("Oligo accuracy:",
#       rf_random.cv_results_['mean_test_Oligo_accuracy'][rf_random.best_index_]*100)

# print("Others accuracy:",
#       rf_random.cv_results_['mean_test_Others_accuracy'][rf_random.best_index_]*100)


# print("Classified A as N:",
#       rf_random.cv_results_['mean_test_A_as_N'][rf_random.best_index_]*100)

# print("Classified A as O:",
#       rf_random.cv_results_['mean_test_A_as_O'][rf_random.best_index_]*100)

# print("Classified A as Others:",
#       rf_random.cv_results_['mean_test_A_as_Others'][rf_random.best_index_]*100)

# print("Classified N as A:",
#       rf_random.cv_results_['mean_test_N_as_A'][rf_random.best_index_]*100)

# print("Classified N as O:",
#       rf_random.cv_results_['mean_test_N_as_O'][rf_random.best_index_]*100)

# print("Classified N as Others:",
#       rf_random.cv_results_['mean_test_N_as_Others'][rf_random.best_index_]*100)

# print("Classified O as A:",
#       rf_random.cv_results_['mean_test_O_as_A'][rf_random.best_index_]*100)

# print("Classified O as N:",
#       rf_random.cv_results_['mean_test_O_as_N'][rf_random.best_index_]*100)

# print("Classified O as Others:",
#       rf_random.cv_results_['mean_test_O_as_Others'][rf_random.best_index_]*100)


# print("Classified Others as A:",
#       rf_random.cv_results_['mean_test_Others_as_A'][rf_random.best_index_]*100)

# print("Classified Others as N:",
#       rf_random.cv_results_['mean_test_Others_as_N'][rf_random.best_index_]*100)

# print("Classified Others as O:",
#       rf_random.cv_results_['mean_test_Others_as_O'][rf_random.best_index_]*100)
                                                       

False:
PR-AUC: 80.92402569869004
ROC-AUC: 95.57920955659537
ROC-AUC: 94.42973203783929
Balanced accuracy: 79.2362046769988
F1_weighted: 80.27764195252072
Astrocyte accuracy: 82.05882352941177
Neuron accuracy: 82.56894790602655
Oligo accuracy: 78.29272151898734
Others accuracy: 74.02432575356954
Classified A as N: 5.555555555555556
Classified A as O: 7.908496732026143
Classified A as Others: 4.477124183006537
Classified N as A: 14.259448416751788
Classified N as O: 0.45454545454545453
Classified N as Others: 2.7170582226762003
Classified O as A: 9.859177215189874
Classified O as N: 0.12658227848101267
Classified O as Others: 11.721518987341772
Classified Others as A: 7.771020624008461
Classified Others as N: 7.810682178741408
Classified Others as O: 10.393971443680591

## Manual cross validation, using PR curves

In [33]:
### manual cross-validation method 
x= X_train
y=y_train
## Setting up cross-validation 
skf = StratifiedKFold(n_splits=10) # shuffling = False, no need to set random_state
skf.get_n_splits(x,y) # using only training data
print(skf)

#for train_index, test_index in skf.split(x,y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("Train_size:", train_index.size, "Test_size:", test_index.size)
    
## Training 
accuracies = []
accuracies_c = []

t_accuracies= []
t_accuracies_c= []

reports = []
reports_c = []

t_reports= []
t_reports_c= []

confusion_matrices = []
confusion_matrices_c = [] 

t_confusion_matrices=[]
t_confusion_matrices_c=[]

#train_features =[] 
#train_n_features=[]
y_preds = []
y_preds_c = []

y_preds_t =[]
y_preds_t_c =[]

y_prob_preds = []
y_prob_preds_c = []

y_cv_test=[]
x_cv_test=[]

roc_auc_scores=[]
roc_auc_scores_c=[]

log_losses=[]
log_losses_c=[]

#brier_scores=[]
#brier_scores_c=[]

best_parameters=[]
best_parameters_c=[]

for train_index, test_index in skf.split(x,y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    #print("Train_size:", train_index.size, "Test_size:", test_index.size)
    x_train_, x_test_ = x.iloc[train_index], x.iloc[test_index]
    y_train_, y_test_ = y[train_index], y[test_index]
    x_test_l = X_train_l.iloc[test_index]
    #print("n of each cell types at test:\n", y_test_.value_counts())
    
    ## 1) Create classifier 
    
    model=Pipeline([
    ('normalizer',MinMaxScaler()),
    ('selector',RFE(SVC(kernel='linear'),n_features_to_select=36)), #cv=5, =30
        #cv=3
    ('clf', BalancedRandomForestClassifier(random_state= 42, sampling_strategy='all', 
                                           n_estimators= 1800, min_samples_split= 2,
                                           min_samples_leaf= 1, max_features= 'auto',
                                           max_depth= 60, bootstrap= False))
                                           ### MAKE SURE THESE ARE CORRECT 
        
        #with alpha, sampling strategy
#        {'selector__n_features_to_select': 36, 'clf__sampling_strategy': 'all', 'clf__random_state': 42,
#         'clf__n_estimators': 1800, 'clf__min_samples_split': 2, 'clf__min_samples_leaf': 1,
#         'clf__max_features': 'auto', 'clf__max_depth': 60, 'clf__ccp_alpha': 0.0, 'clf__bootstrap': False}
        
        
])

    # 2) Train the calibrated classifier with 'training' data (x,y) 
    model.fit(x_train_,y_train_)
    
    # 3) Get class probability predictions for 'test' data 
    y_prob_predict = model.predict_proba(x_test_)
    
    # 4.1) For thresholding: convert y_test_ from name classes to numeric classes
    y_test_numeric = name_to_numeric_classes(y_test_)
    
    # 4.2) For thresholding: use predicted class probabilities to calculate ROC curve for each class vs rest
    
    precision,recall,thresh = multiclass_PR_curves(4,y_test_numeric,y_prob_predict)
    
    # 4.3) For thresholding: from ROC curves, find the best location (fpr,tpr,thresh) for each class 
    # Evaluated based on g-mean 
    best_params_ = best_param_f_score(4,precision,recall,thresh)
    best_parameters.append(best_params_)
    
    # 4.4) For thresholding: apply thresholding to each class to create crisp class label 
    t_class = threshold_list_of_classes_5(y_prob_predict,best_params_)
    
    # 5) Get class labels using default thresholding value (0.5)
    y_predict = model.predict(x_test_)
    
    # 6) Put predictions (labels &probabilities & t_labels) in the corresponding list
    y_preds.append(y_predict)
    
    y_prob_preds.append(y_prob_predict)
    
    y_preds_t.append(t_class)
    
    y_cv_test.append(y_test_) # for visualisation purposes later on
    x_cv_test.append(x_test_l)
    
    # 7) Remove 'ambiguous class' from t_class & y_test_ - for accuracy calculation
    (y_predict_no_amb,y_test_no_amb) = remove_amb_class(t_class,y_test_)
    
    # 8) Calculate and put Performance metric (balanced accuracy) per fold into a list
    accuracies.append(balanced_accuracy_score(y_test_,y_predict)) ## using BALANCED ACC.
    
    t_accuracies.append(balanced_accuracy_score(y_test_no_amb,y_predict_no_amb))
    
    #8.1) Compute classification reports
    reports.append(classification_report(y_test_,y_predict,output_dict=True)) 
  #  reports_c.append(classification_report(y_test_,y_predict_c,output_dict=True)) 
    
    t_reports.append(classification_report(y_test_no_amb,y_predict_no_amb,output_dict=True))
    
    # 9) Calculate and put ROC AUC scores per fold into a list 
    roc_auc_scores.append(roc_auc_score(y_test_,y_prob_predict,multi_class='ovr',average='weighted'))
    
    #9.1) Calculate and put log loss per fold into a list
    log_losses.append(log_loss(y_test_,y_prob_predict))

    
    # 10) Create confusion matrices for default & thresholded results per fold then put in a list 
    cm = confusion_matrix(y_test_,y_predict, labels=["Astro","Neuron","Oligo","Others"]) #,normalize='true'
    
    cm_t = confusion_matrix(y_test_no_amb,y_predict_no_amb, labels=["Astro","Neuron","Oligo","Others"])#,normalize='true'
    
    confusion_matrices.append(cm)
    
    t_confusion_matrices.append(cm_t)

print('mean ROC AUC:',mean(roc_auc_scores))
print('--------------------------------')
print('mean log loss:',mean(log_losses))
print('--------------------------------')

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
mean ROC AUC: 0.9571905149600852
--------------------------------
mean log loss: 0.5317302788856928
--------------------------------


Extracting information from best parameters

In [34]:
#Thresholds 
astro_t = [] 
neuron_t=[]
oligo_t=[]
others_t=[]
#G-means
astro_gm=[]
neuron_gm=[]
oligo_gm=[]
others_gm=[]

#Info extraction 
for fold in best_parameters:
    a_t = fold[0][0]
    n_t = fold[1][0]
    o_t = fold[2][0]
    ot_t = fold[3][0]
    
    a_gm = fold[0][1]
    n_gm = fold[1][1]
    o_gm = fold[2][1]
    ot_gm = fold[3][1]
    
    astro_t.append(a_t)
    neuron_t.append(n_t)
    oligo_t.append(o_t)
    others_t.append(ot_t)
    
    astro_gm.append(a_gm)
    neuron_gm.append(n_gm)
    oligo_gm.append(o_gm)
    others_gm.append(ot_gm)

In [35]:
print("NON CALIBRATED")
print('mean astro threshold:', mean(astro_t), ', mean f1_macro:',mean(astro_gm))
print('mean neuron threshold:', mean(neuron_t), ', mean f1_macro:',mean(neuron_gm))
print('mean oligo threshold:', mean(oligo_t), ', mean f1_macro:',mean(oligo_gm))
print('mean others threshold:', mean(others_t), ', mean f1_macro:',mean(others_gm))
print(mean(astro_t)+mean(neuron_t)+mean(oligo_t)+mean(others_t)) #is it okay that it is above 1? 

NON CALIBRATED
mean astro threshold: 0.5700555555555555 , mean f1_macro: 0.6145856278064705
mean neuron threshold: 0.33294444444444443 , mean f1_macro: 0.9139441941756347
mean oligo threshold: 0.32716666666666666 , mean f1_macro: 0.8737302506308778
mean others threshold: 0.3637222222222222 , mean f1_macro: 0.8022441895278665
1.593888888888889


**NO Thresholding:** Non-calibrated

In [ ]:
# #Confusion matrix across 10 folds, WITHOUT thresholding 
# print('with no thresholding:',mean(accuracies)*100)
# print('Macro avg F1 ',mean([i['macro avg']['f1-score'] for i in reports])*100)
# print('Weighted avg F1 ',mean([i['weighted avg']['f1-score'] for i in reports])*100)

# print("--------------------------")
# C=sum(confusion_matrices)
# final_cm =  C.astype('float') / C.sum(axis=1)[:, np.newaxis]*100 #normalize(sum(confusion_matrices))*100
# print(C)
# print(final_cm)
# print("--------------------------")
# print("Astro accuracy",final_cm[0][0])
# print("Neuron accuracy",final_cm[1][1])
# print("Oligo accuracy",final_cm[2][2])
# print("Others accuracy",final_cm[3][3])
# print("--------------------------")
# # F1-score per class: 
# print('Astro f1-score ',mean([i['Astro']['f1-score'] for i in reports])*100)
# print('Neuron f1-score ',mean([i['Neuron']['f1-score'] for i in reports])*100)
# print('Oligo f1-score ',mean([i['Oligo']['f1-score'] for i in reports])*100)
# print('Others f1-score ',mean([i['Others']['f1-score'] for i in reports])*100)
# print("--------------------------")
# print('Macro avg precision',mean([i['macro avg']['precision'] for i in reports])*100)
# print('Macro avg recall ',mean([i['macro avg']['recall'] for i in reports])*100)

**NO Thresholding:** calibrated

In [ ]:
# #Confusion matrix across 10 folds, WITHOUT thresholding 
# print('with no thresholding, calibrated ACC:',mean(accuracies_c)*100)
# print('Macro avg F1 ',mean([i['macro avg']['f1-score'] for i in reports_c])*100)
# print('Weighted avg F1 ',mean([i['weighted avg']['f1-score'] for i in reports_c])*100)
# print("--------------------------")
# C=sum(confusion_matrices_c)
# final_cm =  C.astype('float') / C.sum(axis=1)[:, np.newaxis]*100 #normalize(sum(confusion_matrices))*100
# print(C)
# print(final_cm)
# print("--------------------------")
# print("Astro accuracy",final_cm[0][0])
# print("Neuron accuracy",final_cm[1][1])
# print("Oligo accuracy",final_cm[2][2])
# print("Others accuracy",final_cm[3][3])
# print("--------------------------")
# # F1-score per class: 
# print('Astro f1-score ',mean([i['Astro']['f1-score'] for i in reports_c])*100)
# print('Neuron f1-score ',mean([i['Neuron']['f1-score'] for i in reports_c])*100)
# print('Oligo f1-score ',mean([i['Oligo']['f1-score'] for i in reports_c])*100)
# print('Others f1-score ',mean([i['Others']['f1-score'] for i in reports_c])*100)
# print("--------------------------")
# print('Macro avg precision',mean([i['macro avg']['precision'] for i in reports_c])*100)
# print('Macro avg recall ',mean([i['macro avg']['recall'] for i in reports_c])*100)

**Thresholding:** Non-calibrated

In [40]:
#0.5 
#Confusion matrix across 10 folds, WITH thresholding 
print('with thresholding (non-calibrated) ACC :',mean(t_accuracies)*100)
print('Macro avg F1 ',mean([i['macro avg']['f1-score'] for i in t_reports])*100)
print('Weighted avg F1 ',mean([i['weighted avg']['f1-score'] for i in t_reports])*100)
print("--------------------------")
C_t=sum(t_confusion_matrices)
final_cm_t =  C_t.astype('float') / C_t.sum(axis=1)[:, np.newaxis]*100
print(C_t)
print(final_cm_t)
print("--------------------------")
print("Astro accuracy",final_cm_t[0][0])
print("Neuron accuracy",final_cm_t[1][1])
print("Oligo accuracy",final_cm_t[2][2])
print("Others accuracy",final_cm_t[3][3])
print('------------------------------')
# F1-score per class: 
print('Astro f1-score ',mean([i['Astro']['f1-score'] for i in t_reports])*100)
print('Astro precision ',mean([i['Astro']['precision'] for i in t_reports])*100)
print('Astro recall ',mean([i['Astro']['recall'] for i in t_reports])*100)
print("--------------------------")
print('Neuron f1-score ',mean([i['Neuron']['f1-score'] for i in t_reports])*100)
print('Neuron precision ',mean([i['Neuron']['precision'] for i in t_reports])*100)
print('Neuron recall ',mean([i['Neuron']['recall'] for i in t_reports])*100)
print("--------------------------")
print('Oligo f1-score ',mean([i['Oligo']['f1-score'] for i in t_reports])*100)
print('Oligo precision ',mean([i['Oligo']['precision'] for i in t_reports])*100)
print('Oligo recall ',mean([i['Oligo']['recall'] for i in t_reports])*100)
print("--------------------------")
print('Others f1-score ',mean([i['Others']['f1-score'] for i in t_reports])*100)
print('Others precision ',mean([i['Others']['precision'] for i in t_reports])*100)
print('Others recall ',mean([i['Others']['recall'] for i in t_reports])*100)
print("--------------------------")
print('Macro avg precision',mean([i['macro avg']['precision'] for i in t_reports])*100)
print('Macro avg recall ',mean([i['macro avg']['recall'] for i in t_reports])*100)

# Checking on disagreements 


thresholded_preds = pd.concat([pd.DataFrame(i) for i in y_preds_t])
thresholded_preds = thresholded_preds.rename(columns={0:'t_Class'})
thresholded_preds = thresholded_preds.reset_index(drop=True)


preds = pd.concat([pd.DataFrame(i) for i in y_preds])
preds = preds.rename(columns={0:'Class'})
preds = preds.reset_index(drop=True)

truth = pd.concat([pd.DataFrame(i) for i in y_cv_test])
truth = truth.rename(columns={'Class':'Truth'})
truth = truth.reset_index(drop=True)

# x_truth = pd.concat([pd.DataFrame(i[['Image','Centroid_X','Centroid_Y']]) for i in x_cv_test])

#Combine absolute prediction to thresholded prediction

#get predicted probabilities
p_probs=pd.concat([pd.DataFrame(i) for i in y_prob_preds])
p_probs= p_probs.rename(columns={0:'Astro',1:'Neuron',2:'Oligo',3:'Others'})
p_probs = p_probs.reset_index(drop=True)

results = thresholded_preds.copy()
results.loc[:,'Class']=preds
results.loc[:,'Truth'] = truth
results.loc[:,'Astro'] = p_probs['Astro']
results.loc[:,'Neuron'] = p_probs['Neuron']
results.loc[:,'Oligo'] = p_probs['Oligo']
results.loc[:,'Others'] = p_probs['Others']
# results.loc[:,'Image'] = x_truth['Image']
# results.loc[:,'Centroid_X'] = x_truth['Centroid_X']
# results.loc[:,'Centroid_Y'] = x_truth['Centroid_Y']


#Calculate agreement between the two 
results.loc[:,'agreement'] = (results['t_Class']==results['Class'])*1
agreements = results['agreement'].value_counts()
print('Agreement: ',agreements[1],'/',agreements[1]+agreements[0],'=> ',(agreements[1]/(agreements[1]+agreements[0])*100,'%') )
print('Disagreement: ',agreements[0],'/',agreements[1]+agreements[0],'=> ',(agreements[0]/(agreements[1]+agreements[0])*100,'%') )
print('------------------------------')
# Of those disagreed, what are they? (those with prob < 0.5)
print('Of the disagreements, what are they?')
disagreed = results[results['agreement']==0]
disagreed['Class'].value_counts()

with thresholding (non-calibrated) ACC : 86.62040920073237
Macro avg F1  84.45393181277237
Weighted avg F1  89.15483466055369
--------------------------
[[120  15   7  10]
 [ 47 758   1  16]
 [ 24   0 627  38]
 [ 14  20  46 423]]
[[78.94736842  9.86842105  4.60526316  6.57894737]
 [ 5.71776156 92.21411192  0.1216545   1.94647202]
 [ 3.48330914  0.         91.00145138  5.51523948]
 [ 2.7833002   3.97614314  9.14512922 84.09542744]]
--------------------------
Astro accuracy 78.94736842105263
Neuron accuracy 92.21411192214111
Oligo accuracy 91.00145137880988
Others accuracy 84.09542743538768
------------------------------
Astro f1-score  67.70262703312568
Astro precision  59.36722689075631
Astro recall  79.9140989729225
--------------------------
Neuron f1-score  93.7921007501118
Neuron precision  95.69018452882193
Neuron recall  92.07723123213515
--------------------------
Oligo f1-score  91.17474966549538
Oligo precision  91.89491568477203
Oligo recall  90.6836019030072
----------------

Astro     168
Others    135
Oligo      61
Neuron     48
Name: Class, dtype: int64

In [41]:
thresholded_preds['t_Class'].value_counts()

Neuron       793
Oligo        681
Others       487
Ambiguous    307
Astro        205
Name: t_Class, dtype: int64

In [42]:
results.head()

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement
0,Neuron,Neuron,Neuron,0.005000,0.991667,0.000000,0.003333,1
1,Neuron,Neuron,Neuron,0.002222,0.996667,0.000000,0.001111,1
2,Oligo,Oligo,Oligo,0.082222,0.008333,0.828333,0.081111,1
3,Neuron,Neuron,Neuron,0.025556,0.942222,0.003889,0.028333,1
4,Oligo,Oligo,Oligo,0.005556,0.000000,0.935556,0.058889,1


In [43]:
results[results['t_Class']!=results['Truth']]

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement
9,Others,Others,Oligo,0.090000,0.050556,0.305000,0.554444,1
10,Astro,Astro,Oligo,0.785556,0.013333,0.181667,0.019444,1
11,Neuron,Neuron,Astro,0.169444,0.656111,0.011667,0.162778,1
13,Ambiguous,Neuron,Neuron,0.091667,0.489444,0.013333,0.405556,0
21,Astro,Astro,Others,0.766667,0.082778,0.110000,0.040556,1
...,...,...,...,...,...,...,...,...
2466,Astro,Astro,Oligo,0.742222,0.148889,0.053889,0.055000,1
2468,Others,Others,Neuron,0.218889,0.155000,0.031111,0.595000,1
2470,Ambiguous,Others,Oligo,0.019444,0.003889,0.187222,0.789444,0
2471,Others,Others,Oligo,0.047778,0.010000,0.096667,0.845556,1


In [44]:
x_test =pd.concat(x_cv_test)
x_test=x_test.reset_index(drop=True)
x_test_subset=x_test[['Image','Centroid_X','Centroid_Y']]

In [45]:
results_=results.copy()
results_=results.reset_index(drop=True)

In [46]:
results_ = results_.join(x_test_subset)

In [47]:
results_.head()

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement,Image,Centroid_X,Centroid_Y
0,Neuron,Neuron,Neuron,0.005000,0.991667,0.000000,0.003333,1,703484.svs,4916.2,10545.9
1,Neuron,Neuron,Neuron,0.002222,0.996667,0.000000,0.001111,1,703484.svs,4933.9,10550.1
2,Oligo,Oligo,Oligo,0.082222,0.008333,0.828333,0.081111,1,703484.svs,4912.0,10557.6
3,Neuron,Neuron,Neuron,0.025556,0.942222,0.003889,0.028333,1,703484.svs,4789.5,10559.3
4,Oligo,Oligo,Oligo,0.005556,0.000000,0.935556,0.058889,1,703484.svs,4829.5,10563.8


In [48]:
results_['Image'].value_counts()

755472.svs    247
703484.svs    246
755480.svs    245
721735.svs    231
755485.svs    228
722215.svs    215
755525.svs    198
721856.svs    181
755481.svs    179
755486.svs    178
721771.svs    168
721701.svs    157
Name: Image, dtype: int64

In [52]:
i = results_[results_['Image']=='703484.svs']
i[i['Truth']!=i['t_Class']]

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement,Image,Centroid_X,Centroid_Y
9,Others,Others,Oligo,0.090000,0.050556,0.305000,0.554444,1,703484.svs,5179.2,10567.4
10,Astro,Astro,Oligo,0.785556,0.013333,0.181667,0.019444,1,703484.svs,5018.0,10568.8
11,Neuron,Neuron,Astro,0.169444,0.656111,0.011667,0.162778,1,703484.svs,4958.4,10568.8
13,Ambiguous,Neuron,Neuron,0.091667,0.489444,0.013333,0.405556,0,703484.svs,4756.0,10573.3
21,Astro,Astro,Others,0.766667,0.082778,0.110000,0.040556,1,703484.svs,4697.9,10581.6
37,Ambiguous,Others,Neuron,0.097778,0.154444,0.067222,0.680556,0,703484.svs,4969.8,10599.5
38,Ambiguous,Oligo,Oligo,0.082222,0.002222,0.468889,0.446667,0,703484.svs,5412.2,10559.8
42,Oligo,Oligo,Others,0.004444,0.000556,0.912222,0.082778,1,703484.svs,5359.6,10585.8
43,Ambiguous,Others,Others,0.220556,0.158889,0.069444,0.551111,0,703484.svs,5348.4,10586.2
47,Neuron,Neuron,Astro,0.032222,0.818333,0.007222,0.142222,1,703484.svs,3562.3,10558.0


In [56]:
incorrect = results_[results_['t_Class']!=results_['Truth']]
incorrect.head()

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement,Image,Centroid_X,Centroid_Y
9,Others,Others,Oligo,0.090000,0.050556,0.305000,0.554444,1,703484.svs,5179.2,10567.4
10,Astro,Astro,Oligo,0.785556,0.013333,0.181667,0.019444,1,703484.svs,5018.0,10568.8
11,Neuron,Neuron,Astro,0.169444,0.656111,0.011667,0.162778,1,703484.svs,4958.4,10568.8
13,Ambiguous,Neuron,Neuron,0.091667,0.489444,0.013333,0.405556,0,703484.svs,4756.0,10573.3
21,Astro,Astro,Others,0.766667,0.082778,0.110000,0.040556,1,703484.svs,4697.9,10581.6


In [57]:
incorrect_as_file = incorrect[['Image','Truth','Centroid_X','Centroid_Y','t_Class']]
path_ = 'D:/Tanrada_classification/imbalance_cortical_training/cortical_full_slide_predictions/Probabilistic_classification/Model8_corrected_PR/Cell_inspection/incorrect.txt'
incorrect_as_file.to_csv(path_, sep='\t',index=False)

In [174]:
correct = results_[results_['t_Class']==results_['Truth']]
correct.head()

,t_Class,Class,Truth,Astro,Neuron,Oligo,Others,agreement,Image,Centroid_X,Centroid_Y
0,Neuron,Neuron,Neuron,0.005000,0.991667,0.000000,0.003333,1,703484.svs,4916.2,10545.9
1,Neuron,Neuron,Neuron,0.002222,0.996667,0.000000,0.001111,1,703484.svs,4933.9,10550.1
2,Oligo,Oligo,Oligo,0.082222,0.008333,0.828333,0.081111,1,703484.svs,4912.0,10557.6
3,Neuron,Neuron,Neuron,0.025556,0.942222,0.003889,0.028333,1,703484.svs,4789.5,10559.3
4,Oligo,Oligo,Oligo,0.005556,0.000000,0.935556,0.058889,1,703484.svs,4829.5,10563.8


In [175]:
correct_as_file = correct[['Image','Truth','Centroid_X','Centroid_Y','t_Class']]
path_ = 'D:/Tanrada_classification/imbalance_cortical_training/cortical_full_slide_predictions/Probabilistic_classification/Model8_corrected_PR/Cell_inspection/correct.txt'
correct_as_file.to_csv(path_, sep='\t',index=False)

**Thresholding:** Calibrated

In [134]:
#0.5 
#Confusion matrix across 10 folds, WITH thresholding 
print('with thresholding (calibrated) ACC:',mean(t_accuracies_c)*100)
print('Macro avg F1 ',mean([i['macro avg']['f1-score'] for i in t_reports_c])*100)
print('Weighted avg F1 ',mean([i['weighted avg']['f1-score'] for i in t_reports_c])*100)
print("--------------------------")
C_t=sum(t_confusion_matrices_c)
final_cm_t =  C_t.astype('float') / C_t.sum(axis=1)[:, np.newaxis]*100
print(C_t)
print(final_cm_t)
print("--------------------------")
print("Astro accuracy",final_cm_t[0][0])
print("Neuron accuracy",final_cm_t[1][1])
print("Oligo accuracy",final_cm_t[2][2])
print("Others accuracy",final_cm_t[3][3])
print('------------------------------')
# F1-score per class: 
print('Astro f1-score ',mean([i['Astro']['f1-score'] for i in t_reports_c])*100)
print('Neuron f1-score ',mean([i['Neuron']['f1-score'] for i in t_reports_c])*100)
print('Oligo f1-score ',mean([i['Oligo']['f1-score'] for i in t_reports_c])*100)
print('Others f1-score ',mean([i['Others']['f1-score'] for i in t_reports_c])*100)
print("--------------------------")
print('Macro avg precision',mean([i['macro avg']['precision'] for i in t_reports_c])*100)
print('Macro avg recall ',mean([i['macro avg']['recall'] for i in t_reports_c])*100)
# Checking on disagreements 
thresholded_preds = pd.concat([pd.DataFrame(i) for i in y_preds_t_c])
thresholded_preds = thresholded_preds.rename(columns={0:'t_Class'})

preds = pd.concat([pd.DataFrame(i) for i in y_preds_c])
preds = preds.rename(columns={0:'Class'})

#Combine absolute prediction to thresholded prediction
results = thresholded_preds.copy()
results.loc[:,'Class']=preds

#Calculate agreement between the two 
results.loc[:,'agreement'] = (results['t_Class']==results['Class'])*1
agreements = results['agreement'].value_counts()
print('Agreement: ',agreements[1],'/',agreements[1]+agreements[0],'=> ',(agreements[1]/(agreements[1]+agreements[0])*100,'%') )
print('Disagreement: ',agreements[0],'/',agreements[1]+agreements[0],'=> ',(agreements[0]/(agreements[1]+agreements[0])*100,'%') )
print('------------------------------')
# Of those disagreed, what are they? (those with prob < 0.5)
print('Of the disagreements, what are they?')
disagreed = results[results['agreement']==0]
disagreed['Class'].value_counts()

with thresholding (calibrated) ACC: 85.36068938459995
Macro avg F1  83.74867031298476
Weighted avg F1  88.22661256240006
--------------------------
[[115  22   8   8]
 [ 42 782   1  17]
 [ 20   1 648  56]
 [ 14  25  54 445]]
[[75.16339869 14.37908497  5.22875817  5.22875817]
 [ 4.98812352 92.87410926  0.11876485  2.01900238]
 [ 2.75862069  0.13793103 89.37931034  7.72413793]
 [ 2.60223048  4.64684015 10.03717472 82.71375465]]
--------------------------
Astro accuracy 75.16339869281046
Neuron accuracy 92.87410926365796
Oligo accuracy 89.37931034482759
Others accuracy 82.71375464684016
------------------------------
Astro f1-score  67.93304850274961
Neuron f1-score  93.59111059295563
Oligo f1-score  89.8708065290597
Others f1-score  83.5997156271741
--------------------------
Macro avg precision 83.18000797391211
Macro avg recall  85.36068938459995
Agreement:  2170 / 2473 =>  (87.74767488879903, '%')
Disagreement:  303 / 2473 =>  (12.252325111200971, '%')
------------------------------
O

Others    94
Astro     72
Oligo     69
Neuron    68
Name: Class, dtype: int64